# Payload Analytics with Watson OpenScale

This notebook should be run in a Watson Studio project, using Default Python 3.6 runtime environment. It requires service credentials and a Cloud API key to access the following Cloud services:
* Watson OpenScale

The notebook will gather and visualize payload records from Watson OpenScale for the German Credit Risk model. The notebook assumes the model has been created/deployed to Watson Machine Learning and that the subscription has been created in Watson OpenScale.

#### Dependency Setup

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

In [ ]:
import pandas as pd
import json
import time

from datetime import datetime
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.utils import get_instance_guid

#### Configure Service Credentials

Update the cells below with your Cloud API Key.

In [ ]:
CLOUD_API_KEY = "PASTE HERE"

#### Model Parameters

__Ensure that the two parameters match the model / deployment you have previously subscribed__

In [ ]:
MODEL_NAME = "Spark German Risk Model"
DEPLOYMENT_NAME = "Spark German Risk Deployment"

#### Get Watson OpenScale GUID
Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.

In [ ]:
wos_client = None
WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print("Watson OpenScale GUID: {}".format(WOS_GUID))

wos_client = APIClient(aios_credentials=WOS_CREDENTIALS)
print("Watson OpenScale Python Client Version: {}".format(wos_client.version))

#### Get subscription

We have previously subscribed Watson OpenScale to our machine learning model. Here we get that subscription.

In [ ]:
wos_client.data_mart.subscriptions.list()

subscriptions_uids = wos_client.data_mart.subscriptions.get_uids()
subscription_id = None
for sub in subscriptions_uids:
    if wos_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
        subscription = wos_client.data_mart.subscriptions.get(sub)
        subscription_id = sub
        break
            
if subscription is None:
    print('Subscription not found.')

print("Subscription ID: {}".format(subscription_id))

## Payload analytics

#### Get data distributions as pandas dataframe


In [ ]:
start_date = "2018-01-01T00:00:00.00Z"
end_date = datetime.utcnow().isoformat() + "Z"

sex_distribution = subscription.payload_logging.data_distribution.run(
            start_date=start_date,
            end_date=end_date,
            group=['prediction', 'Sex'],
            agg=['count'])

In [ ]:
sex_distribution_run_uid = sex_distribution['id']
distributions_pd = subscription.payload_logging.data_distribution.get_run_result(run_id=sex_distribution_run_uid, format='pandas')
distributions_pd

In [ ]:
subscription.payload_logging.data_distribution.show_chart(sex_distribution_run_uid);

In [ ]:
credit_history_distribution = subscription.payload_logging.data_distribution.run(
            start_date=start_date,
            end_date=end_date,
            group=['prediction', 'CreditHistory'],
            agg=['count'])

In [ ]:
credit_history_distribution_run_uid = credit_history_distribution['id']

subscription.payload_logging.data_distribution.show_chart(credit_history_distribution_run_uid);


## Credits
This notebook was adapted from the following sources:

* [Monitor Models Code Pattern](https://github.com/IBM/monitor-wml-model-with-watson-openscale)
* [OpenScale Labs](https://github.com/pmservice/OpenScale-Labs)
* [OpenScale Tutorials](https://github.com/pmservice/ai-openscale-tutorials)

#### Original Authors
* Eric Martens, is a technical specialist having expertise in analysis and description of business processes, and their translation into functional and non-functional IT requirements. He acts as the interpreter between the worlds of IT and business.
* Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.